<a href="https://colab.research.google.com/github/felixsimard/comp551-p4/blob/main/MNIST_MLP_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [9]:
for ims, labels in train_loader:
    im = ims[0:10]
    break
im.size()
im.flatten(1).size()

torch.Size([10, 784])

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense = nn.Sequential(
            nn.Linear(784, 100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(100, 10),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        x = self.dense(x)
        return x
    

In [46]:
for data, labels in train_loader:
    bz, c, h , w = data.size()
    data = data.resize(bz, c, h * w, 1)
    break
data.size()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


torch.Size([20, 1, 784, 1])

In [4]:
model = MLP()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [7]:
# number of epochs to train the model
n_epochs = 5  # suggest training between 20-50 epochs

model.train() # prep model for training

for epoch in range(n_epochs):
    train_loss = 0.0

    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.088418
Epoch: 2 	Training Loss: 0.075125
Epoch: 3 	Training Loss: 0.063370
Epoch: 4 	Training Loss: 0.051529
Epoch: 5 	Training Loss: 0.042358


In [8]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.091315

Test Accuracy of     0: 98% (964/980)
Test Accuracy of     1: 99% (1124/1135)
Test Accuracy of     2: 97% (1007/1032)
Test Accuracy of     3: 97% (983/1010)
Test Accuracy of     4: 96% (951/982)
Test Accuracy of     5: 96% (858/892)
Test Accuracy of     6: 96% (929/958)
Test Accuracy of     7: 97% (999/1028)
Test Accuracy of     8: 95% (932/974)
Test Accuracy of     9: 97% (988/1009)

Test Accuracy (Overall): 97% (9735/10000)
